In [2]:

from facenet_pytorch import  InceptionResnetV1, training
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from tensorboardX  import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [3]:
#Checking and loading CUDA if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [4]:
#Transforming and loading data

trans = transforms.Compose([
    # np.float32,
    transforms.Resize((200,200)),  # Resize images to 32x32 to fit the CNN input size # Convert to grayscale
    transforms.ToTensor(),  # Convert image to tensor
    # transforms.Grayscale(1),
     transforms.Normalize((0.5), (0.5))  # Normalize
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # fixed_image_standardization
])

# load the training and test sets
dataset = datasets.ImageFolder(
    root='Dataset',
    transform = trans
)
print(dataset.classes)

['Abdullah Munir', 'Aizaz', 'Ali riaz', 'Anas', 'Anousha', 'Ayaz', 'Fahad', 'Faizan', 'Farhan', 'Hammad', 'Hannan', 'Haris', 'Hirar', 'Irfan', 'Kashmala', 'Khadija', 'Kumail', 'Maryam', 'Moiz', 'Moiz SaP', 'Moizullah', 'Mustafa', 'Noor', 'Obaid', 'Raheel', 'Saqib', 'Sufyan', 'Talal', 'Tayyaba', 'Usaid', 'Zeeshan', 'Zehra', 'ahmed', 'ahsan', 'asad', 'basit', 'bilal', 'gopi', 'musaddiq', 'muzammil', 'omer', 'osama', 'owais', 'rafay', 'rafia', 'shehroz', 'shezil', 'sohaib', 'subhan', 'vinay', 'zain', 'zainab', 'zamran', 'zayan']


In [5]:
# Define the split ratio
train_size = int(.9* len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing

# Split the dataset
trainset, testset = random_split(dataset, [train_size, test_size])

In [6]:
# InceptionResnetV1 Model
from vit_pytorch import SimpleViT
models={}
model = InceptionResnetV1(
    classify=True,# ?
    pretrained='vggface2',
    num_classes=len(dataset.classes)
)
# model.conv2d_1a = nn.Conv2d(
#     in_channels=3,  # Change to 1 for grayscale images
#     out_channels=32,  # Keep the same number of output channels
#     kernel_size=3,
#     stride=2,
#     padding=0,
#     bias=False
# )
model=model.to(device)
models["InceptionResnetV1"]=model
model = SimpleViT(
        image_size = 200,
        channels=3,
        patch_size = 20,
        num_classes = len(dataset.classes),
        dim = 512,
        depth = 6,
        heads = 16,
        mlp_dim = 1024
    )

model=model.to(device)

models["VIT"]=model

In [7]:
models["VIT"]

SimpleViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=20, p2=20)
    (1): LayerNorm((1200,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=1200, out_features=512, bias=True)
    (3): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (transformer): Transformer(
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0-5): 6 x ModuleList(
        (0): Attention(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (attend): Softmax(dim=-1)
          (to_qkv): Linear(in_features=512, out_features=3072, bias=False)
          (to_out): Linear(in_features=1024, out_features=512, bias=False)
        )
        (1): FeedForward(
          (net): Sequential(
            (0): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=512, out_features=1024, bias=True)
            (2): GELU(approximate='none')
  

In [8]:
# Create DataLoaders
trainloader = DataLoader(trainset, batch_size=8, shuffle=True)
testloader = DataLoader(testset, batch_size=8, shuffle=True)

metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [9]:
models["InceptionResnetV1"]


InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [10]:
#Function to save checkpoints
#To allow you to resume training from the last saved point, avoiding the need to start over if something interrupts the training. This helps save time and effort.

def save_checkpoint(model, optimizer, epoch, file_path='checkpoint.pth'):
    checkpoint = {
        'model_state_dict': model.state_dict(),#state_dict() is a dictionary that stores all the learnable parameters (like weights and biases) and buffers of a model or optimizer in PyTorch.
        'optimizer_state_dict': optimizer.state_dict(),#optimizer.state_dict() stores the current state of the optimizer, including parameters like learning rates and momentum, allowing you to resume training with the same optimization settings.
        'epoch': epoch,#Saves the current training epoch (the number of times the model has seen the entire dataset).
    }
    
    torch.save(checkpoint, file_path)
    print(f'Checkpoint saved at {file_path}')

In [11]:
from PIL import Image
for data in testloader:
    img, label = data
    print(img.mode)

TypeError: expected np.ndarray (got numpy.ndarray)

In [10]:
#Function to load checkpoints
def load_checkpoint(model, optimizer=None, file_path='checkpoint.pth'):
    print(file_path)
    checkpoint = torch.load(file_path)
    print("here")
    # print(checkpoint)
    model.load_state_dict(checkpoint['model_state_dict'])
    print("here working......")
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    epoch = checkpoint['epoch']
    
    print(f'Checkpoint loaded from {file_path} (Epoch: {epoch})')
    
    return model, optimizer, epoch

In [11]:
def transfer_learning(model,model_name,file_path):
    try:

        checkpoint = torch.load(file_path)

        # Step 1: Modify the classification layer for transfer learning
        if model_name=='InceptionResnetV1':
            model.logits = nn.Linear(in_features=model.logits.in_features,out_features=len(dataset.classes) , bias=True)
    
        elif model_name=='VIT':
            model.linear_head = nn.Linear(in_features=model.linear_head.in_features, out_features=len(dataset.classes), bias=True)
            # Remove the `linear_head` weights from the checkpoint to avoid size mismatch
            del checkpoint['model_state_dict']['linear_head.weight']
            del checkpoint['model_state_dict']['linear_head.bias']
            
        # Load the remaining weights
        model.load_state_dict(checkpoint['model_state_dict'], strict=False)
        model.to(device)
        return model
    except Exception as ex:
        print(f"No previous learning found. Training model from scratch {ex}")
        return model

In [12]:
# model, optimizer, start_epoch= load_checkpoint(model, optimizer, 'checkpoint.pth')
# model = model.to(device)
# total_epochs = start_epoch
# model = init_model()
# model = model.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# start_epoch=0
# total_epochs = 0
# print("new model loaded")

In [13]:
#Optimizer, scheduler and loss func
print("here")
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])
loss_fn = torch.nn.CrossEntropyLoss()

here


In [17]:
num_epochs = 15
import tqdm 

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in tqdm.tqdm(trainloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        
        images, labels = images.to(device), labels.to(device)
#         print(images.shape())
        optimizer.zero_grad()  # Zero the parameter gradients
        
        outputs = model(images)  # Forward pass
        loss = loss_fn(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize the model parameters
        
        running_loss += loss.item() * images.size(0)  # Update running loss
        
        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(trainset)
    epoch_train_accuracy = 100 * correct_train / total_train

    # Evaluate on test set
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    
    test_accuracy = 100 * correct_test / total_test

    # Print loss and accuracy
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_train_accuracy:.2f}%")
    print(f"Test Accuracy: {test_accuracy:.2f}%")

Epoch 1/15: 100%|██████████| 1565/1565 [01:10<00:00, 22.04it/s]


Epoch [1/15]
Train Loss: 0.1622, Train Accuracy: 95.59%
Test Accuracy: 98.13%


Epoch 2/15: 100%|██████████| 1565/1565 [01:10<00:00, 22.08it/s]


Epoch [2/15]
Train Loss: 0.1045, Train Accuracy: 97.16%
Test Accuracy: 99.86%


Epoch 3/15: 100%|██████████| 1565/1565 [01:10<00:00, 22.33it/s]


...

Epoch [8/15]
Train Loss: 0.0419, Train Accuracy: 98.95%
Test Accuracy: 99.93%


Epoch 9/15: 100%|██████████| 1565/1565 [01:10<00:00, 22.25it/s]


Epoch [9/15]
Train Loss: 0.0635, Train Accuracy: 98.49%
Test Accuracy: 99.78%


Epoch 10/15:  39%|███▊      | 604/1565 [00:27<00:43, 22.26it/s]


KeyboardInterrupt: 

In [29]:
import cv2
import cv2
import numpy as np
import torch

model.eval()
correct_test = 0
total_test = 0
classes = dataset.classes

def tensor_to_numpy(image_tensor):
    # Convert tensor to numpy array
    image_np = image_tensor.cpu().numpy()
    # Transpose from CHW to HWC
    image_np = np.transpose(image_np, (1, 2, 0))
    # Ensure values are in the range [0, 255]
    image_np = (image_np * 255).astype(np.uint8)
    return image_np

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        
        # For displaying individual images
        for i in range(images.size(0)):
            # Convert tensor to numpy array and then to RGB format
            singleImage = tensor_to_numpy(images[i])
            # Display the image using OpenCV
            cv2.imshow(f"{classes+1}", singleImage)
            cv2.waitKey(0)  # Wait for a key press to close the image window

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Close all OpenCV windows
cv2.destroyAllWindows()

# Print loss and accuracy
print(f"Epoch [{epoch+1}/{num_epochs}]")
print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")

QObject::moveToThread: Current thread (0x5fd9e13b1e10) is not the object's thread (0x5fd9d2adf430).
Cannot move to target thread (0x5fd9e13b1e10)

QObject::moveToThread: Current thread (0x5fd9e13b1e10) is not the object's thread (0x5fd9d2adf430).
Cannot move to target thread (0x5fd9e13b1e10)

QObject::moveToThread: Current thread (0x5fd9e13b1e10) is not the object's thread (0x5fd9d2adf430).
Cannot move to target thread (0x5fd9e13b1e10)

QObject::moveToThread: Current thread (0x5fd9e13b1e10) is not the object's thread (0x5fd9d2adf430).
Cannot move to target thread (0x5fd9e13b1e10)

QObject::moveToThread: Current thread (0x5fd9e13b1e10) is not the object's thread (0x5fd9d2adf430).
Cannot move to target thread (0x5fd9e13b1e10)

QObject::moveToThread: Current thread (0x5fd9e13b1e10) is not the object's thread (0x5fd9d2adf430).
Cannot move to target thread (0x5fd9e13b1e10)

QObject::moveToThread: Current thread (0x5fd9e13b1e10) is not the object's thread (0x5fd9d2adf430).
Cannot move to tar

In [12]:
# Apply transfer learning modifications 
key='InceptionResnetV1'

#load the previous model if exist otherwise initialize the new model
def get_checkpoint_path(key):
    return f'checkpoint_{key}.pth'

try:
    model, optimizer, start_epoch= load_checkpoint(model, optimizer, get_checkpoint_path(key=key))
    model = model.to(device)
    total_epochs = start_epoch
except Exception as  ex   :
    start_epoch=0
    total_epochs = 0
    print(f"new model loaded:{ex}")
    
model = transfer_learning(models[key],key,get_checkpoint_path(key=key))
model.to(device)


checkpoint_InceptionResnetV1.pth
here
new model loaded:Error(s) in loading state_dict for SimpleViT:
	Missing key(s) in state_dict: "to_patch_embedding.1.weight", "to_patch_embedding.1.bias", "to_patch_embedding.2.weight", "to_patch_embedding.2.bias", "to_patch_embedding.3.weight", "to_patch_embedding.3.bias", "transformer.norm.weight", "transformer.norm.bias", "transformer.layers.0.0.norm.weight", "transformer.layers.0.0.norm.bias", "transformer.layers.0.0.to_qkv.weight", "transformer.layers.0.0.to_out.weight", "transformer.layers.0.1.net.0.weight", "transformer.layers.0.1.net.0.bias", "transformer.layers.0.1.net.1.weight", "transformer.layers.0.1.net.1.bias", "transformer.layers.0.1.net.3.weight", "transformer.layers.0.1.net.3.bias", "transformer.layers.1.0.norm.weight", "transformer.layers.1.0.norm.bias", "transformer.layers.1.0.to_qkv.weight", "transformer.layers.1.0.to_out.weight", "transformer.layers.1.1.net.0.weight", "transformer.layers.1.1.net.0.bias", "transformer.layers.1.1.

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [28]:
model.eval()
training.pass_epoch(
    model, loss_fn, testloader,
    batch_metrics=metrics, show_running=True, device=device,
)

y_pred: tensor([[ 0.4113, -1.1935,  0.1879, -2.0930,  1.1690,  1.3743,  0.8994, -1.3153,
         -1.4898,  0.4697, -1.3059,  1.8529, -1.1033, -0.2418, -1.2389, -0.0630,
          2.3832, -0.5623, -2.3759,  1.1048, -0.5108,  1.7299,  1.1599,  0.3757,
          4.2055, -1.0222,  0.9584, -3.0609, -1.6618, -2.7106,  0.7606, -0.5793,
         -2.0833,  0.7736, -0.8790, -0.3149,  5.7074,  3.8838, -1.4655,  2.6487,
          3.7230, -1.2543,  0.4363, -0.7337,  2.7063, -1.2809,  1.4963, -0.6300,
          2.8131, -1.3236, -1.2974, -2.5439, -1.6770,  0.4997],
        [ 0.5181, -0.9673, -1.3167, -1.4378,  1.8919,  1.2724,  1.7006,  0.1885,
         -1.1261,  0.4086, -1.0080, -0.3698,  0.5955, -0.9303,  1.9881,  2.1888,
          2.8500, -1.9302, -2.0699,  0.1576, -2.5893,  2.3901,  2.3221,  0.1215,
          4.1556, -3.7291, -0.6588, -2.3292, -1.6275, -1.8598, -0.1388,  0.0389,
         -0.6280, -0.0159,  4.5640, -1.9780,  4.5466, -1.5073, -1.4126,  7.0041,
         -3.4945,  0.8991, -0.5539, -

(tensor(1.7669), {'fps': tensor(283.3683), 'acc': tensor(0.5126)})

In [43]:
#Training loop with tensorboard summary writer
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10
epochs = 100
# try:
#     # print("working")
#     model, optimizer, start_epoch= load_checkpoint(model, optimizer, 'checkpoint.pth')
#     # print("working")
#     model.to(device)
#     total_epochs = start_epoch
#     # print("working")
# except:
#     start_epoch=0
#     total_epochs = 0
#     print("Starting Training from Scratch")


print(f'\n\nModel:{key} before Training')
print('-' * 10)
model.eval()
training.pass_epoch(
    model, loss_fn, testloader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

# epoch = start_epoch


for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    #training loop
    model.train()
    training.pass_epoch(
        model, loss_fn, trainloader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    #testing loop
    model.eval()
    training.pass_epoch(
        model, loss_fn, testloader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )
    total_epochs += 1
    save_checkpoint(model, optimizer, total_epochs, file_path=get_checkpoint_path(key))
writer.close()



Model:InceptionResnetV1 before Training
----------
Valid |   140/140  | loss:    0.0071 | fps:  441.7005 | acc:    0.9979   

Epoch 1/100
----------
Train |   418/418  | loss:    0.0143 | fps:  239.5096 | acc:    0.9976   
Valid |   140/140  | loss:    0.0061 | fps:  460.0532 | acc:    0.9979   
Checkpoint saved at checkpoint_InceptionResnetV1.pth

Epoch 2/100
----------
Train |   418/418  | loss:    0.0146 | fps:  241.8082 | acc:    0.9978   
Valid |   140/140  | loss:    0.0059 | fps:  441.0877 | acc:    0.9986   
Checkpoint saved at checkpoint_InceptionResnetV1.pth

Epoch 3/100
----------
Train |   418/418  | loss:    0.0145 | fps:  244.5096 | acc:    0.9976   
Valid |   140/140  | loss:    0.0071 | fps:  468.3460 | acc:    0.9979   
Checkpoint saved at checkpoint_InceptionResnetV1.pth

Epoch 4/100
----------
Train |   418/418  | loss:    0.0159 | fps:  246.5050 | acc:    0.9968   
Valid |   140/140  | loss:    0.0057 | fps:  469.7130 | acc:    0.9979   
Checkpoint saved at checkp

KeyboardInterrupt: 

In [39]:
model, optimizer, start_epoch= load_checkpoint(model, optimizer, get_checkpoint_path(key=key))
print(start_epoch)
model.eval()

here
here working......
Checkpoint loaded from checkpoint_InceptionResnetV1.pth (Epoch: 3)
3


InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [42]:
dataset.classes

['Abdullah Munir',
 'Aizaz',
 'Ali riaz',
 'Anas',
 'Anousha',
 'Ayaz',
 'Fahad',
 'Faizan',
 'Farhan',
 'Hammad',
 'Hannan',
 'Haris',
 'Hirar',
 'Irfan',
 'Kashmala',
 'Khadija',
 'Kumail',
 'Maryam',
 'Moiz',
 'Moiz SaP',
 'Moizullah',
 'Mustafa',
 'Noor',
 'Obaid',
 'Raheel',
 'Saqib',
 'Sufyan',
 'Talal',
 'Tayyaba',
 'Usaid',
 'Zeeshan',
 'Zehra',
 'ahmed',
 'ahsan',
 'asad',
 'basit',
 'bilal',
 'gopi',
 'musaddiq',
 'muzammil',
 'omer',
 'osama',
 'owais',
 'rafay',
 'rafia',
 'shehroz',
 'shezil',
 'sohaib',
 'subhan',
 'vinay',
 'zain',
 'zainab',
 'zamran',
 'zayan']

In [57]:
#Testing the model on real videos

import cv2
import torch
from torchvision import transforms
from PIL import Image


class_names = dataset.classes

# Capture video from webcam
cap = cv2.VideoCapture('testing_videos/subhan.mp4')  # Change to 'video.mp4' for video file
from facenet_pytorch import MTCNN

mtcnn = MTCNN(keep_all=True,image_size= 200, min_face_size = 20, thresholds=[0.9,0.9,0.9],post_process = True, device='cuda' if torch.cuda.is_available() else 'cpu')


# Define the transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),  # Resize images to 200x200
    transforms.ToTensor(),  # Convert image to tensor
    #transforms.Grayscale(3),
    transforms.Normalize((0.5,), (0.5,))  # Normalize with mean and std for grayscale
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

while True:
    ret, frame = cap.read()
    # print("here")
    if not ret:
        # print("here")
        break
    # print("here")
    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect faces
    boxes, _ = mtcnn.detect(rgb_frame)
    # print(boxes)
    # print("hello")
    
    if boxes is not None:
        for box in boxes:
#             print("hello")
            x1, y1, x2, y2 = map(int, box)
            
            # Extract face ROI
            face = frame[y1:y2, x1:x2]
            if face.size == 0:
                continue
            
            try:
                # Apply transformations
                face_pil = Image.fromarray(face)  # Convert numpy array to PIL image
                face_transformed = transform(face_pil)  # Apply transformations
                face_transformed = face_transformed.unsqueeze(0)  # Add batch dimension
#                 print("hello")
            except Exception as e:
                print(f"Error during preprocessing: {e}")
#                 print("hello")
                continue
            
            # Move the tensor to the device (GPU/CPU)
            face_transformed = face_transformed.to(device)
            
            # Classify face
            with torch.no_grad():
                print(face_transformed.shape)
                output = model(face_transformed)
                _, predicted = torch.max(output, 1)
                label = predicted.item()
            
            label_name = class_names[label]
            # Draw rectangle and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, str(label_name), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Video', frame)
    # print("hello")
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release capture and close windows
cap.release()
cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x60cd3f4cc820) is not the object's thread (0x60cd4273b7d0).
Cannot move to target thread (0x60cd3f4cc820)

QObject::moveToThread: Current thread (0x60cd3f4cc820) is not the object's thread (0x60cd4273b7d0).
Cannot move to target thread (0x60cd3f4cc820)

QObject::moveToThread: Current thread (0x60cd3f4cc820) is not the object's thread (0x60cd4273b7d0).
Cannot move to target thread (0x60cd3f4cc820)

QObject::moveToThread: Current thread (0x60cd3f4cc820) is not the object's thread (0x60cd4273b7d0).
Cannot move to target thread (0x60cd3f4cc820)

QObject::moveToThread: Current thread (0x60cd3f4cc820) is not the object's thread (0x60cd4273b7d0).
Cannot move to target thread (0x60cd3f4cc820)

QObject::moveToThread: Current thread (0x60cd3f4cc820) is not the object's thread (0x60cd4273b7d0).
Cannot move to target thread (0x60cd3f4cc820)

QObject::moveToThread: Current thread (0x60cd3f4cc820) is not the object's thread (0x60cd4273b7d0).
Cannot move to tar

torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1, 3, 200, 200])
torch.Size([1,

In [28]:
# #Converting model to ONNX

# from torch.autograd import Variable
# from mtcnn_ort import MTCNN
# import torch.onnx
# import torch

# dummy_input = Variable(torch.randn(1, 1, 200, 200)).to(device)
# print(dummy_input.device)
# checkpoint = torch.load('checkpoint.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
# torch.onnx.export(model, dummy_input, "model.onnx")
# torch.onnx.export(mtcnn, dummy_input, "model_mtcnn.onnx")
